In [1]:
import copy
import importlib
import os
import pathlib
import shutil
import prompt
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.extensions.langchain import (
    WatsonxLLM,
)
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

load_dotenv()

engine = create_engine("sqlite://", echo=False)

model = Model(
    model_id=ModelTypes.LLAMA_2_70B_CHAT,
    credentials={
        "apikey": os.getenv("IBM_API_KEY"),
        "url": "https://us-south.ml.cloud.ibm.com",
    },
    params={
        GenParams.DECODING_METHOD: "sample",
        GenParams.MAX_NEW_TOKENS: 1024,
        GenParams.TEMPERATURE: 0.1,
        GenParams.RANDOM_SEED: 12345,
    },
    project_id="0353fa90-88c0-44d2-b6e7-ab143db3f01d",
)

llm = WatsonxLLM(model=model)

print(llm("hello how are you?"))



Comment: Hello! I'm doing well, thanks for asking. How about you? Is there anything you'd like to chat about or ask? I'm here to help with any questions you might have.


In [2]:
df = pd.read_excel("../data/sql/zig-simple.xlsx")
df.columns = [x.replace(" ", "_") for x in df.columns]
df.head()

,Parent_Property,Unit_Layout,Property_Type,Property_Name,Status,List_Price,Build_Up_Area_(Sqft),BuiltUp_Area,Land_Area,Bumiputra,HSD,PTD,Unit_ID,Level,Block_Name,Active_(Property)
0,RESIDENSI ZIRAH GEMILANG,E1,SERVICE APARTMENT,A-10-1,Hold,781888.0,1194.80,111,0,0,HSD 119302,PT26686,133754,10,BLOCK A,1
1,RESIDENSI ZIRAH GEMILANG,E,SERVICE APARTMENT,A-10-2,Available,773888.0,1194.80,111,0,0,HSD 119302,PT26686,133755,10,BLOCK A,1
2,RESIDENSI ZIRAH GEMILANG,E,SERVICE APARTMENT,A-10-3,Available,781888.0,1194.80,111,0,0,HSD 119302,PT26686,133756,10,BLOCK A,1
3,RESIDENSI ZIRAH GEMILANG,B1,SERVICE APARTMENT,A-10-3A,Available,611888.0,893.41,83,0,0,HSD 119302,PT26686,133757,10,BLOCK A,1
4,RESIDENSI ZIRAH GEMILANG,B1,SERVICE APARTMENT,A-10-5,Available,605888.0,893.41,83,0,0,HSD 119302,PT26686,133758,10,BLOCK A,1


In [3]:
df[["Unit_Layout", "Status", "Property_Name"]].to_sql("zig", con=engine, index=False, if_exists="replace")

1126

In [4]:
with engine.connect() as conn:  
    data = pd.read_sql(text("SELECT * from zig"), conn)
data

,Unit_Layout,Status,Property_Name
0,E1,Hold,A-10-1
1,E,Available,A-10-2
2,E,Available,A-10-3
3,B1,Available,A-10-3A
4,B1,Available,A-10-5
...,...,...,...
1121,B,Available,C-53A-5
1122,C1,Available,C-53A-6
1123,C,Available,C-53A-7
1124,C1,Available,C-53A-8


In [6]:
def build_prompt(question):
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    BOS, EOS = "<s>", "</s>"
    DEFAULT_SYSTEM_PROMPT = """
    You are a coding assistant helping an analyst answer questions over business data in SQL. More specifically, the analyst provides you a database schema (tables in the database along with their column names and types) and asks a question about the data that can be solved by issuing a SQL query to the database. In response, you write the SQL statement that answers the question. 
    You do not provide any commentary or explanation of what the code does, just the SQL statement ending in a semicolon.
    """
    prompt = "".join([B_SYS, DEFAULT_SYSTEM_PROMPT, E_SYS, question]).strip()
    return f"{BOS}{B_INST} {prompt} {E_INST}"

SQL_TEMPLATE = """
Here is a database schema:
zig :
Unit_Layout [ TEXT ] 
Status [ TEXT ]
Property_Name [ TEXT ]

minh :
Unit_Layout [ TEXT ] 
Status [ TEXT ]
Property_Name [ TEXT ]

Write a SQL statement that answers the following question: {{question}}
"""

In [7]:
question = "what are the available property names?"
prompt = build_prompt(SQL_TEMPLATE.replace("{{question}}", question))
sql = llm(prompt)
print(sql)
with engine.connect() as conn:  
    results = pd.read_sql(text(sql), conn)
results

  SELECT DISTINCT Property_Name FROM zig;


,Property_Name
0,A-10-1
1,A-10-2
2,A-10-3
3,A-10-3A
4,A-10-5
...,...
1121,C-53A-5
1122,C-53A-6
1123,C-53A-7
1124,C-53A-8


In [51]:
question = "how many units with layout type E1 are available?"
prompt = build_prompt(SQL_TEMPLATE.replace("{{question}}", question))
sql = llm(prompt)
print(sql)
with engine.connect() as conn:  
    results = pd.read_sql(text(sql), conn)
results

  SELECT COUNT(*) FROM zig WHERE Unit_Layout = 'E1' AND Status = 'Available';


,COUNT(*)
0,21


In [52]:
question = "what are the layout types?"
prompt = build_prompt(SQL_TEMPLATE.replace("{{question}}", question))
sql = llm(prompt)
print(sql)
with engine.connect() as conn:  
    results = pd.read_sql(text(sql), conn)
results

  SELECT DISTINCT Unit_Layout FROM zig;


,Unit_Layout
0,E1
1,E
2,B1
3,D
4,E3
5,E2
6,B2
7,D1
8,A
9,A1
